In [1]:
import json 
from openai_api import OPENAI_API
from concurrent.futures import ThreadPoolExecutor, as_completed
import sys
import os

In [2]:
samples = {}

In [3]:
with open("data/ibm_gpt-4o.json") as file:
    data = json.load(file)

In [4]:
data = data[:50] # Just pick 50 samples

In [5]:
# Expanding data by duplicating each sample, one with 'human_code' and one with 'machine_code'
splitted_data = []

for sample in data:
    # Sample with human_code
    human_sample = {key: sample[key] for key in sample if key not in ['machine_code', 'human_code']}
    human_sample["code"] = sample["human_code"]
    human_sample["writer"] = "human"
    
    # Sample with machine_code
    machine_sample = {key: sample[key] for key in sample if key not in ['machine_code', 'human_code']}
    machine_sample["code"] = sample["machine_code"]
    machine_sample["writer"] = "machine"
    
    # Add both samples to the expanded_data list
    splitted_data.extend([human_sample, machine_sample])

In [6]:
# data_h = [{key: sample[key] for key in sample if key != "machine_code"} for sample in data]
# data_m = [{key: sample[key] for key in sample if key != "human_code"} for sample in data]

In [ ]:
os.environ["OPENAI_API_KEY"]

In [8]:
api = OPENAI_API(model='gpt-4o', temperature=1)

In [9]:
rewrtied_data = []

In [10]:
def process_item(item, idx):
    
    # pcode = "```python def main(): for a in range(1, 10):)```"
    code = item['code']
    rewrite_prompt = f"Please first explain the functionality of the Python code below. Then generate a possible rewrite for this Python code according to your explanation. Please just give me a pure code in reponse, not any explanation or text. Please do not add any clarifications after the rewritten code.\n CODE: \n {code}"

    # Generating the rewrite codes
    rewrite_codes = api.communication(prompt=rewrite_prompt, n=3)
    # CleanUp the generated codes
    rewrite_codes = [api.codeSnippetCleanup(item) for item in rewrite_codes]
    item["rewrite"] = rewrite_codes
    return item


In [ ]:
with ThreadPoolExecutor(max_workers=8) as executer:
    
    futures_to_items = {executer.submit(process_item, item, idx): item for idx, item in enumerate(splitted_data)}
    
    for i, future in enumerate(as_completed(futures_to_items)):
        try:
            result = future.result()
            if result is not None: rewrtied_data.append(result)
            sys.stdout.write(f"\rProcessed {i + 1} of {len(splitted_data)} prompts...")
            sys.stdout.flush()
        except Exception as e:
            print(f"\nError processing prompt {i + 1}: {e}")
        # if i == 1 : break

Processed 71 of 100 prompts...

In [ ]:
with open("data/ibm_gpt-4o_rewrtie.json", 'w') as file:
    json.dump(rewrtied_data, file, indent=4)